## **2. Tokenizacja zbiorów danych**

### **Uwaga!**
Ze względu na poufność danych, surowe raporty i adnotacje ekspertów nie są zawarte w tym repozytorium.

### **Model bazowy:** speakleash/Bielik-1.5B-v3 (https://huggingface.co/speakleash/Bielik-1.5B-v3)

#### **Import Bibliotek**

In [15]:
import os
import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict

Ustawienie sztywnych parametrów dla okna kontekstowego zgodnie z dokumentacją modelu Bielik-1.5B-v3 oraz narzucenie wymiaru okna przesuwnego.

In [16]:
MODEL_NAME = "speakleash/Bielik-1.5B-v3"
MAX_LENGTH = 8192
STRIDE = 1024

In [17]:
df_train = pd.read_json('data/data_split/train.jsonl', lines=True)
df_val = pd.read_json('data/data_split/validation.jsonl', lines=True)
df_test = pd.read_json('data/data_split/test.jsonl', lines=True)

#### **2.1. Przygotowanie do tokenizacji.**

In [18]:
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

Wczytanie odpowiedniego tokenizera

In [19]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#### **2.2. Przeprowadzenie tokenizacji.**

In [20]:
def chunk_and_tokenize(examples):
    # Tokenizujemy tekst, co tworzy wiele fragmentów dla długich dokumentów
    tokenized_output = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=False,
    )

    # Pobieramy mapowanie z nowych, podzielonych fragmentów do oryginalnych przykładów
    sample_mapping = tokenized_output.pop("overflow_to_sample_mapping")
    
    # Tworzymy listę etykiet dla każdego nowego fragmentu
    labels = []
    for i in range(len(tokenized_output["input_ids"])):
        original_sample_idx = sample_mapping[i]
        labels.append(examples["labels"][original_sample_idx])
    
    tokenized_output["labels"] = labels
    return tokenized_output

In [21]:
tokenized_datasets = raw_datasets.map(
    chunk_and_tokenize,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/279 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/57 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/57 [00:00<?, ? examples/s]

#### **2.3. Zapis wyników tokenizacji.**

In [22]:

OUTPUT_DIR = "data/data_tokenized"
os.makedirs(OUTPUT_DIR, exist_ok=True)

tokenized_datasets.save_to_disk(OUTPUT_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/3475 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/563 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/683 [00:00<?, ? examples/s]